In [16]:
import pandas as pd
import numpy as np
import re
import math
import os
from Levenshtein import distance as lev
import datetime as dt

In [17]:
def detect_number(x):
    pattern = re.compile('^.*\d.*$')
    if re.match(pattern, x) is None:
        return False
    return True

In [18]:
def multiplier(number, string):
    neg = 1
    check_idx = string.index(number) + len(number)
    if (check_idx - len(number))!=0 and string[check_idx - len(number) - 1]=='-':
        neg = -1
    if check_idx>=len(string) or string[check_idx+1] not in ['b', 'm', 'k', 't'] or string[check_idx+1]=='b':
        return [neg, 0]
    elif string[check_idx+1]=='m':
        return [neg, -3]
    elif string[check_idx+1]=='k':
        return [neg, -6]
    elif string[check_idx+1]=='t':
        return [neg, 3]

In [19]:
def transform_number(numeric_string):
    numeric_string = numeric_string.lower()
    if detect_number(numeric_string):
        pattern = re.compile('[\d(\.\d)?]+')
        numbers = re.findall(pattern, numeric_string)
        power = multiplier(numbers[0], numeric_string)
        numbers[0] = round(power[0]*float(numbers[0])*math.pow(10, power[1]), 3)
        return numbers[0]
    else:
        return np.nan

In [20]:
forbes_list = []
col_names = ['rank', 'name', 'country', 'sales', 'profit', 'assets', 'market_value', 'year']
intermediate_data_types_forbes = {'rank': 'Int32', 'name': 'string', 'country': 'string', 'sales': 'string', 
                                  'profit': 'string', 'assets': 'string', 'market_value': 'string', 'year': 'string', 
                                  'name_lower': 'string'}
act_data_types_forbes = {'rank': 'Int32', 'name': 'string', 'country': 'string', 'sales': 'float64', 'profit': 'float64', 
                         'assets': 'float64', 'market_value': 'float64', 'year': 'Int32', 'name_lower': 'string'}
forbes_directory = '../Project/data/forbes/'
for file in os.listdir(forbes_directory):
    dot_index = file.index('.')
    hyphen_index = file.index('-')
    if file[dot_index:]=='.xlsx':
        df = pd.read_excel(forbes_directory+file)
    elif file[dot_index:]=='.csv':
        df = pd.read_csv(forbes_directory+file)
    if 'year' not in list(map(str.lower, df.columns)):
        df['year'] = file[hyphen_index+1:dot_index]
    df.columns = list(map(lambda x: x.strip().replace(' ', '_').lower() if x!='global company' else 'name', 
                          list(df.columns)))
    df['name_lower'] = df.name.str.lower()
    df = df.astype(intermediate_data_types_forbes)
    transform_columns = ['sales', 'profit', 'assets', 'market_value']
    for column in transform_columns:
        df[column] = df[column].apply(lambda x: transform_number(x) if pd.notna(x) else np.nan)
    df = df.astype(act_data_types_forbes)
    forbes_list.append(df)

In [21]:
forbes = pd.concat(forbes_list, axis=0)
print(forbes.info())
forbes.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12100 entries, 0 to 1999
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   rank          11981 non-null  Int32  
 1   name          12100 non-null  string 
 2   country       12098 non-null  string 
 3   sales         12093 non-null  float64
 4   profit        12097 non-null  float64
 5   assets        12086 non-null  float64
 6   market_value  12092 non-null  float64
 7   year          12100 non-null  Int32  
 8   name_lower    12100 non-null  string 
dtypes: Int32(2), float64(4), string(3)
memory usage: 874.4 KB
None


,rank,name,country,sales,profit,assets,market_value,year,name_lower
0,1,ICBC,China,151.381,41.984,3473.238,229.805,2017,icbc
1,2,China Construction Bank,China,134.242,34.981,3016.578,200.458,2017,china construction bank
2,3,Berkshire Hathaway,United States,222.935,24.074,620.854,409.908,2017,berkshire hathaway
3,4,JPMorgan Chase,United States,102.494,24.231,2512.986,306.616,2017,jpmorgan chase
4,5,Wells Fargo,United States,97.573,21.938,1943.443,274.385,2017,wells fargo


In [22]:
forbes.to_csv('./data/forbes.csv', index=False)

In [23]:
data_types_security = {'ticker_symbol': 'string', 'security': 'string', 'sec_filings': 'string', 'gics_sector': 'string', 'gics_sub_industry': 'string', 'address_of_headquarters': 'string', 'date_first_added': 'datetime64', 'cik': 'string', 'security_lower': 'string'}
securities = pd.read_csv('../Project/data/nyse/securities.csv', dtype=data_types_security)
securities.columns = list(map(lambda x: x.strip().replace(' ', '_').lower(), list(securities.columns)))
securities['security_lower'] = securities.security.str.lower()
securities = securities.astype(data_types_security)
print(securities.info())
securities.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   ticker_symbol            505 non-null    string        
 1   security                 505 non-null    string        
 2   sec_filings              505 non-null    string        
 3   gics_sector              505 non-null    string        
 4   gics_sub_industry        505 non-null    string        
 5   address_of_headquarters  505 non-null    string        
 6   date_first_added         307 non-null    datetime64[ns]
 7   cik                      505 non-null    string        
 8   security_lower           505 non-null    string        
dtypes: datetime64[ns](1), string(8)
memory usage: 35.6 KB
None


,ticker_symbol,security,sec_filings,gics_sector,gics_sub_industry,address_of_headquarters,date_first_added,cik,security_lower
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",NaT,66740,3m company
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,abbott laboratories
2,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,abbvie
3,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,accenture plc
4,ATVI,Activision Blizzard,reports,Information Technology,Home Entertainment Software,"Santa Monica, California",2015-08-31,718877,activision blizzard


In [24]:
merged_fn_inner = forbes.merge(securities, how='inner', left_on='name_lower', right_on='security_lower', suffixes=['_f', '_s'])
print(merged_fn_inner.info())
print(merged_fn_inner.name.unique().size)
merged_fn_inner.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 873 entries, 0 to 872
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   rank                     868 non-null    Int32         
 1   name                     873 non-null    string        
 2   country                  873 non-null    string        
 3   sales                    872 non-null    float64       
 4   profit                   873 non-null    float64       
 5   assets                   873 non-null    float64       
 6   market_value             873 non-null    float64       
 7   year                     873 non-null    Int32         
 8   name_lower               873 non-null    string        
 9   ticker_symbol            873 non-null    string        
 10  security                 873 non-null    string        
 11  sec_filings              873 non-null    string        
 12  gics_sector              873 non-nul

,rank,name,country,sales,profit,assets,market_value,year,name_lower,ticker_symbol,security,sec_filings,gics_sector,gics_sub_industry,address_of_headquarters,date_first_added,cik,security_lower
0,3,Berkshire Hathaway,United States,222.935,24.074,620.854,409.908,2017,berkshire hathaway,BRK.B,Berkshire Hathaway,reports,Financials,Multi-Sector Holdings,"Omaha, Nebraska",2010-02-16,1067983,berkshire hathaway
1,4,Berkshire Hathaway,United States,235.165,39.742,702.651,491.888,2018,berkshire hathaway,BRK.B,Berkshire Hathaway,reports,Financials,Multi-Sector Holdings,"Omaha, Nebraska",2010-02-16,1067983,berkshire hathaway
2,26,Berkshire Hathaway,United States,247.837,4.021,707.794,516.407,2019,berkshire hathaway,BRK.B,Berkshire Hathaway,reports,Financials,Multi-Sector Holdings,"Omaha, Nebraska",2010-02-16,1067983,berkshire hathaway
3,4,Berkshire Hathaway,United States,254.600,81.400,817.700,455.400,2020,berkshire hathaway,BRK.B,Berkshire Hathaway,reports,Financials,Multi-Sector Holdings,"Omaha, Nebraska",2010-02-16,1067983,berkshire hathaway
4,3,Berkshire Hathaway,United States,245.500,42.500,873.700,624.400,2021,berkshire hathaway,BRK.B,Berkshire Hathaway,reports,Financials,Multi-Sector Holdings,"Omaha, Nebraska",2010-02-16,1067983,berkshire hathaway


In [25]:
forbes_companies_np = forbes.loc[~forbes['name_lower'].isin(merged_fn_inner['name_lower']), ['name_lower']]
securities_companies_np = securities.loc[~securities['security_lower'].isin(merged_fn_inner['security_lower']), 
                                         ['security_lower', 'ticker_symbol']]

In [26]:
merged_fn_cross = forbes_companies_np.merge(securities_companies_np, how='cross')
print(merged_fn_cross.shape)
merged_fn_cross.head()

(3648775, 3)


,name_lower,security_lower,ticker_symbol
0,icbc,3m company,MMM
1,icbc,accenture plc,ACN
2,icbc,acuity brands inc,AYI
3,icbc,adobe systems inc,ADBE
4,icbc,aes corp,AES


In [27]:
def check_company(company_one, company_two):
    company_one_split = company_one.strip().split()
    company_two_split = company_two.strip().split()
    eliminations = ['group', 'inc', 'corp', 'plc', 'llc', 'ban', 'of', 'comp', 'nation', 'invest', 'ltd', 'tech', 
                    'unite', 'west', 'ind', 'syst', 'holdi', 'co', 'finan', 'elec', 'powe', 'inte', 'and', '&', 'amer', 
                    'pharm', 'capi', 'petro', 'prope', 'heal', 'union', 'trust', 'ener', 'resour', 'south', 'resear', 'lim',
                    'the', 'red', 'n.v.', 'exp', 'firs', 'gen', 'info', 'serv', 'eng', 'sol', 'royal', 'enter', 'publ', 
                    'mater']
    for company_name in company_one_split:
        elim = sum(list(map(lambda x: company_name.startswith(x), eliminations)))
        if elim==0 and company_name in company_two_split:
            return True
    return False

In [28]:
merged_fn_cross['levenshtein'] = merged_fn_cross.apply(lambda x: lev(x['name_lower'], x['security_lower']), axis=1)

In [29]:
merged_fn_cross_lev_min = merged_fn_cross.loc[merged_fn_cross.groupby('name_lower')['levenshtein'].idxmin()]

In [30]:
merged_fn_cross_lev_min['same_flag'] = merged_fn_cross_lev_min.apply(lambda x: check_company(x['name_lower'], 
                                                                                             x['security_lower']), axis=1)

In [31]:
merged_companies_same = merged_fn_cross_lev_min.loc[merged_fn_cross_lev_min['same_flag']==True,:]
print(merged_companies_same.shape)
dup_companies = ['royal caribbean group', 'yum brands', 'baker hughes company']
merged_companies_same = merged_companies_same.loc[~merged_companies_same['name_lower'].isin(dup_companies), :]
merged_companies_same.shape
merged_companies_same.head()

(165, 5)


,name_lower,security_lower,ticker_symbol,levenshtein,same_flag
73776,accenture,accenture plc,ACN,4,True
200203,adobe systems,adobe systems inc,ADBE,4,True
306479,aes,aes corp,AES,5,True
42905,aetna,aetna inc,AET,4,True
515131,affiliated managers group,affiliated managers group inc,AMG,4,True


In [32]:
cross_companies = merged_fn_cross[['name_lower', 'security_lower', 'ticker_symbol']]
same_companies = merged_companies_same[['name_lower', 'security_lower', 'ticker_symbol']]
new_merge = pd.merge(cross_companies, same_companies, how='inner', on=['name_lower', 'security_lower', 'ticker_symbol'])

In [33]:
new_merge = new_merge.drop_duplicates()
new_merge

,name_lower,security_lower,ticker_symbol
0,jpmorgan chase,jpmorgan chase & co.,JPM
6,at&t,at&t inc,T
12,citigroup,citigroup inc.,C
18,microsoft,microsoft corp.,MSFT
24,amazon.com,amazon.com inc,AMZN
...,...,...,...
741,quanta services,quanta services inc.,PWR
744,universal health services,"universal health services, inc.",UHS
746,fidelity national information (fis),fidelity national information services,FIS
747,block,block h&r,HRB


In [34]:
forbes_processed = pd.merge(forbes.loc[~forbes['name_lower'].isin(merged_fn_inner['name_lower']), ['name', 'name_lower']], 
                            merged_companies_same[['name_lower', 'security_lower', 'ticker_symbol']], 
                            how='inner', on='name_lower').drop_duplicates()
forbes_processed

,name,name_lower,security_lower,ticker_symbol
0,JPMorgan Chase,jpmorgan chase,jpmorgan chase & co.,JPM
6,AT&T,at&t,at&t inc,T
12,Citigroup,citigroup,citigroup inc.,C
18,Microsoft,microsoft,microsoft corp.,MSFT
24,Amazon.com,amazon.com,amazon.com inc,AMZN
...,...,...,...,...
741,Quanta Services,quanta services,quanta services inc.,PWR
744,Universal Health Services,universal health services,"universal health services, inc.",UHS
746,Fidelity National Information (FIS),fidelity national information (fis),fidelity national information services,FIS
747,Block,block,block h&r,HRB


In [35]:
forbes_processed_full = pd.merge(forbes.loc[~forbes['name_lower'].isin(merged_fn_inner['name_lower']), :], 
                                 merged_companies_same[['name_lower', 'security_lower', 'ticker_symbol']], 
                                 how='inner', on='name_lower')
forbes_processed_full

,rank,name,country,sales,profit,assets,market_value,year,name_lower,security_lower,ticker_symbol
0,4,JPMorgan Chase,United States,102.494,24.231,2512.986,306.616,2017,jpmorgan chase,jpmorgan chase & co.,JPM
1,3,JPMorgan Chase,United States,118.180,26.496,2609.785,387.668,2018,jpmorgan chase,jpmorgan chase & co.,JPM
2,2,JPMorgan Chase,United States,132.912,32.738,2737.188,368.502,2019,jpmorgan chase,jpmorgan chase & co.,JPM
3,3,JPMorgan Chase,United States,142.900,30.000,3.000,291.700,2020,jpmorgan chase,jpmorgan chase & co.,JPM
4,2,JPMorgan Chase,United States,136.200,40.400,3.000,464.800,2021,jpmorgan chase,jpmorgan chase & co.,JPM
...,...,...,...,...,...,...,...,...,...,...,...
744,920,Universal Health Services,United States,11.600,0.941,13.500,12.400,2021,universal health services,"universal health services, inc.",UHS
745,1277,Universal Health Services,United States,12.640,0.990,13.090,9.900,2022,universal health services,"universal health services, inc.",UHS
746,663,Fidelity National Information (FIS),United States,13.880,0.417,82.930,61.050,2022,fidelity national information (fis),fidelity national information services,FIS
747,1002,Block,United States,17.660,0.166,13.930,59.550,2022,block,block h&r,HRB


In [36]:
processed_companies = pd.merge(forbes_processed, 
                               securities.loc[~securities['security_lower'].isin(merged_fn_inner['security_lower']), 
                                              ['security', 'security_lower']], how='inner', on='security_lower')
processed_companies

,name,name_lower,security_lower,ticker_symbol,security
0,JPMorgan Chase,jpmorgan chase,jpmorgan chase & co.,JPM,JPMorgan Chase & Co.
1,AT&T,at&t,at&t inc,T,AT&T Inc
2,Citigroup,citigroup,citigroup inc.,C,Citigroup Inc.
3,Microsoft,microsoft,microsoft corp.,MSFT,Microsoft Corp.
4,Amazon.com,amazon.com,amazon.com inc,AMZN,Amazon.com Inc
...,...,...,...,...,...
158,Xylem,xylem,xylem inc.,XYL,Xylem Inc.
159,Envision Healthcare,envision healthcare,envision healthcare corp,EVHC,Envision Healthcare Corp
160,Quanta Services,quanta services,quanta services inc.,PWR,Quanta Services Inc.
161,Universal Health Services,universal health services,"universal health services, inc.",UHS,"Universal Health Services, Inc."


In [37]:
df = pd.concat([merged_fn_inner[['name', 'security', 'ticker_symbol']], 
                processed_companies[['name', 'security', 'ticker_symbol']]], axis=0).drop_duplicates().sort_values(by='name')
df

,name,security,ticker_symbol
101,AES,AES Corp,AES
1,AT&T,AT&T Inc,T
112,AbbVie,AbbVie,ABBV
196,Abbott Laboratories,Abbott Laboratories,ABT
25,Accenture,Accenture plc,ACN
...,...,...,...
124,Xilinx,Xilinx Inc,XLNX
158,Xylem,Xylem Inc.,XYL
154,Yum! Brands,Yum! Brands Inc,YUM
647,Zions Bancorp,Zions Bancorp,ZION


In [38]:
df.to_csv('./data/companies_forbes_nyse.csv', index=False)

In [39]:
processed_companies_full = pd.merge(forbes_processed_full, 
                                    securities.loc[~securities['security_lower'].isin(merged_fn_inner['security_lower']), :]
                                    , how='inner', on='security_lower', suffixes=['_f', '_s'])
processed_companies_full.drop('ticker_symbol_s', inplace=True, axis=1)
processed_companies_full.rename(columns={'ticker_symbol_f': 'ticker_symbol'}, inplace=True)
processed_companies_full

,rank,name,country,sales,profit,assets,market_value,year,name_lower,security_lower,ticker_symbol,security,sec_filings,gics_sector,gics_sub_industry,address_of_headquarters,date_first_added,cik
0,4,JPMorgan Chase,United States,102.494,24.231,2512.986,306.616,2017,jpmorgan chase,jpmorgan chase & co.,JPM,JPMorgan Chase & Co.,reports,Financials,Banks,"New York, New York",1975-06-30,19617
1,3,JPMorgan Chase,United States,118.180,26.496,2609.785,387.668,2018,jpmorgan chase,jpmorgan chase & co.,JPM,JPMorgan Chase & Co.,reports,Financials,Banks,"New York, New York",1975-06-30,19617
2,2,JPMorgan Chase,United States,132.912,32.738,2737.188,368.502,2019,jpmorgan chase,jpmorgan chase & co.,JPM,JPMorgan Chase & Co.,reports,Financials,Banks,"New York, New York",1975-06-30,19617
3,3,JPMorgan Chase,United States,142.900,30.000,3.000,291.700,2020,jpmorgan chase,jpmorgan chase & co.,JPM,JPMorgan Chase & Co.,reports,Financials,Banks,"New York, New York",1975-06-30,19617
4,2,JPMorgan Chase,United States,136.200,40.400,3.000,464.800,2021,jpmorgan chase,jpmorgan chase & co.,JPM,JPMorgan Chase & Co.,reports,Financials,Banks,"New York, New York",1975-06-30,19617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
744,1194,Quanta Services,United States,11.200,0.446,8.400,13.200,2021,quanta services,quanta services inc.,PWR,Quanta Services Inc.,reports,Industrials,Industrial Conglomerates,"Houston, Texas",2009-07-01,1050915
745,1412,Quanta Services,United States,12.980,0.486,12.860,17.200,2022,quanta services,quanta services inc.,PWR,Quanta Services Inc.,reports,Industrials,Industrial Conglomerates,"Houston, Texas",2009-07-01,1050915
746,920,Universal Health Services,United States,11.600,0.941,13.500,12.400,2021,universal health services,"universal health services, inc.",UHS,"Universal Health Services, Inc.",reports,Health Care,Health Care Facilities,"King of Prussia, Pennsylvania",2014-09-20,352915
747,1277,Universal Health Services,United States,12.640,0.990,13.090,9.900,2022,universal health services,"universal health services, inc.",UHS,"Universal Health Services, Inc.",reports,Health Care,Health Care Facilities,"King of Prussia, Pennsylvania",2014-09-20,352915


In [40]:
df_full = pd.concat([merged_fn_inner, processed_companies_full], axis=0).drop(['name_lower', 'security_lower'], axis=1)
df_full

,rank,name,country,sales,profit,assets,market_value,year,ticker_symbol,security,sec_filings,gics_sector,gics_sub_industry,address_of_headquarters,date_first_added,cik
0,3,Berkshire Hathaway,United States,222.935,24.074,620.854,409.908,2017,BRK.B,Berkshire Hathaway,reports,Financials,Multi-Sector Holdings,"Omaha, Nebraska",2010-02-16,1067983
1,4,Berkshire Hathaway,United States,235.165,39.742,702.651,491.888,2018,BRK.B,Berkshire Hathaway,reports,Financials,Multi-Sector Holdings,"Omaha, Nebraska",2010-02-16,1067983
2,26,Berkshire Hathaway,United States,247.837,4.021,707.794,516.407,2019,BRK.B,Berkshire Hathaway,reports,Financials,Multi-Sector Holdings,"Omaha, Nebraska",2010-02-16,1067983
3,4,Berkshire Hathaway,United States,254.600,81.400,817.700,455.400,2020,BRK.B,Berkshire Hathaway,reports,Financials,Multi-Sector Holdings,"Omaha, Nebraska",2010-02-16,1067983
4,3,Berkshire Hathaway,United States,245.500,42.500,873.700,624.400,2021,BRK.B,Berkshire Hathaway,reports,Financials,Multi-Sector Holdings,"Omaha, Nebraska",2010-02-16,1067983
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
744,1194,Quanta Services,United States,11.200,0.446,8.400,13.200,2021,PWR,Quanta Services Inc.,reports,Industrials,Industrial Conglomerates,"Houston, Texas",2009-07-01,1050915
745,1412,Quanta Services,United States,12.980,0.486,12.860,17.200,2022,PWR,Quanta Services Inc.,reports,Industrials,Industrial Conglomerates,"Houston, Texas",2009-07-01,1050915
746,920,Universal Health Services,United States,11.600,0.941,13.500,12.400,2021,UHS,"Universal Health Services, Inc.",reports,Health Care,Health Care Facilities,"King of Prussia, Pennsylvania",2014-09-20,352915
747,1277,Universal Health Services,United States,12.640,0.990,13.090,9.900,2022,UHS,"Universal Health Services, Inc.",reports,Health Care,Health Care Facilities,"King of Prussia, Pennsylvania",2014-09-20,352915


In [41]:
# df_full.to_csv('./data/forbes_nyse_sec_merged.csv', index=False)

In [42]:
def check_date(x):
    print(x)
    if '-' in x.strip():
        return '-'
    if '/' in x.strip():
        return '/'

In [43]:
nyse_directory = '../Project/data/nyse/csv/'
nyse_list = []
intermediate_nyse_data_types = {'Date': 'string', 'Low': 'float64', 'Open': 'float64', 'High': 'float64', 
                                'Close': 'float64', 'Adjusted Close': 'float64', 'Volume': 'Int64'}
nyse_data_types = {'low': 'float64', 'open': 'float64', 'high': 'float64', 'close': 'float64', 
                   'adjusted_close': 'float64', 'volume': 'Int64', 'ticker_symbol': 'string'}
ticker_symbols = df.ticker_symbol.tolist()
for file in os.listdir(nyse_directory):
    ticker = file[:file.index('.')]
    if sum(list(map(lambda x: len(set(ticker.split('-')) & set(x.split('.'))) >= 1, ticker_symbols))) >= 1:
        ticker_df = pd.read_csv(nyse_directory+file, dtype=intermediate_nyse_data_types)
        ticker_df.columns = list(map(lambda x: x.strip().replace(' ', '_').lower(), list(ticker_df.columns)))
        ticker_df['ticker_symbol'] = ticker
        ticker_df['date'] = pd.to_datetime(ticker_df.date, format='%d-%m-%Y')
        ticker_df = ticker_df.astype(nyse_data_types)
        ticker_df = ticker_df.loc[ticker_df.date.dt.year>=2017, :]
        nyse_list.append(ticker_df)

In [44]:
nyse_prices = pd.concat(nyse_list, axis=0)
print(nyse_prices.info())
nyse_prices.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 208985 entries, 3799 to 10777
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   date            208985 non-null  datetime64[ns]
 1   low             206349 non-null  float64       
 2   open            206349 non-null  float64       
 3   volume          206349 non-null  Int64         
 4   high            206349 non-null  float64       
 5   close           206349 non-null  float64       
 6   adjusted_close  206349 non-null  float64       
 7   ticker_symbol   208985 non-null  string        
dtypes: Int64(1), datetime64[ns](1), float64(5), string(1)
memory usage: 14.5 MB
None


,date,low,open,volume,high,close,adjusted_close,ticker_symbol
3799,2017-01-03,169.309998,170.779999,691300,171.360001,170.600006,161.730347,AAP
3800,2017-01-04,170.369995,170.369995,641700,173.169998,172.000000,163.057556,AAP
3801,2017-01-05,170.229996,170.869995,861000,173.059998,171.880005,162.943756,AAP
3802,2017-01-06,169.300003,171.320007,828000,171.990005,169.630005,160.810791,AAP
3803,2017-01-09,168.320007,170.059998,560100,170.279999,169.529999,160.715942,AAP


In [45]:
nyse_prices.to_csv('./data/nyse_prices.csv', index=False)